# ライブラリのImport, Seedの固定

In [96]:
import random
import os
import torch

import numpy as np
import pandas as pd
import warnings
import pickle
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor
import xgboost as xgb

warnings.simplefilter('ignore')

def seed_torch(seed=42):
    # python の組み込み関数の seed を固定
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # numpy の seed を固定
    np.random.seed(seed)
    # torch の seed を固定
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # 決定論的アルゴリズムを使用する
    torch.backends.cudnn.deterministic = True

SEED = 42
seed_torch(SEED)

# 教師データ読み込みと前処理

In [104]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

df = pd.read_csv("train.csv") 
df_test = pd.read_csv("test.csv") 

#df = df.drop(["Country","City"], axis=1)
#df_test = df_test.drop(["Country","City"], axis=1)

df['date'] = pd.to_datetime({'year': df['year'], 'month': df['month'], 'day': df['day']})
df_test['date'] = pd.to_datetime({'year': df_test['year'], 'month': df_test['month'], 'day': df_test['day']})


df_id = df.copy()
df_test_id = df_test.copy()

df = df.drop(["year","month","day"], axis=1)

In [105]:
#曜日を追加するための関数を定義
def get_weekday_jp(dt):
    w_list = ['月曜日', '火曜日', '水曜日', '木曜日', '金曜日', '土曜日', '日曜日']
    return(w_list[dt.weekday()])

In [106]:
#dateから曜日情報を取得
df["weekday"] = df["date"].apply(get_weekday_jp)
df_test["weekday"] = df_test["date"].apply(get_weekday_jp)

In [107]:
le = LabelEncoder()
df["Country"] = le.fit_transform(df["Country"])
df["City"] = le.fit_transform(df["City"])
df["weekday"] = le.fit_transform(df["weekday"])

df_test["Country"] = le.fit_transform(df_test["Country"])
df_test["City"] = le.fit_transform(df_test["City"])
df_test["weekday"] = le.fit_transform(df_test["weekday"])

# 学習用関数の定義

In [119]:
from dateutil.relativedelta import relativedelta

class TimeSeriesSplitGenerator:
    def __init__(self, n_split = 12, test_day_after = "2019-01-01", slide = False):
        self.test_day_after = pd.to_datetime(test_day_after)
        self.n_split = n_split
        self.test_month_period = 36
        self.month = relativedelta(months = 1)

        self.slide = slide

    def split(self, X):
        for m in range(self.test_month_period):
            test_month = self.test_day_after + relativedelta(months=m)
            if m == 0:
                test_index = (test_month <= X.date) & (X.date < test_month + self.month)  # 2019-1-1＜X＜2019-2-1
                valid_index = (test_month <= X.date) & (X.date < test_month + self.month) # 2019-1-1＜X＜2019-2-1
                train_index =(test_month <= X.date) & (X.date < test_month + self.month)  # 2019-9月よりも前
                if self.slide:
                    train_index = train_index &(
                        test_month - self.month - relativedelta(months =12) <= X.date
                    )
            elif m == 1:
                test_index = (test_month <= X.date) & (X.date < test_month + self.month)  # 2019-2-1＜X＜2019-3-1
                valid_index = (test_month - self.month <= X.date) & (X.date < test_month) # 2019-1-1＜X＜2019-2-1
                train_index = (test_month - self.month <= X.date) & (X.date < test_month) # 2019-9月よりも前
                if self.slide:
                    train_index = train_index &(
                        test_month - self.month - relativedelta(months =12) <= X.date
                    )
            else:
                test_index = (test_month <= X.date) & (X.date < test_month + self.month) # 2019-10   2019-01
                valid_index = (test_month - self.month <= X.date) & (X.date < test_month) # 2019-9
                train_index = (X.date < test_month - self.month)                          # 2019-9月よりも前
                if self.slide:
                    train_index = train_index &(
                        test_month - self.month - relativedelta(months =12) <= X.date
                    )
            yield train_index, valid_index, test_index

In [120]:
def lightgbm(X_train, Y_train, X_valid, Y_valid, X_test):
    bst_params = {
          "boosting_type": "gbdt",
          "metric": "rmse",
          "objective": "regression",
          "n_jobs": -1,
          "seed": SEED,
          'random_state': SEED,
          "learning_rate": 0.01,
          "bagging_fraction": 0.75,
          "bagging_freq": 10,
          "colsample_bytree": 0.75,
          "num_boost_round": 10000,
          "early_stopping_rounds": 10,
          "verbose_eval": 1000,
      }
    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_valid = lgb.Dataset(X_valid, Y_valid)

    model = lgb.train(bst_params, lgb_train,
                        valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],
                        verbose_eval=1000)

    # 検証データに対する予測値を求める
    va_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    #テストデータに対する予測値を求める
    te_pred = np.array(model.predict(X_test, num_iteration=model.best_iteration))

    return va_pred, te_pred, model


def catboost(X_train, Y_train, X_valid, Y_valid, X_test):
    # objectの列番号を取得
    categorical_features_indices = np.where(X_train.dtypes==np.object)[0]
    lgb_train = Pool(X_train, Y_train, cat_features=categorical_features_indices)
    lgb_valid = Pool(X_valid, Y_valid, cat_features=categorical_features_indices)
    model = CatBoostRegressor(eval_metric='RMSE',
                            loss_function='RMSE',
                            num_boost_round=10000,
                            logging_level='Silent',
                            random_seed=SEED)
    model.fit(lgb_train, 
            eval_set=lgb_valid,
            early_stopping_rounds=10,
            verbose=True,
            use_best_model=True)

    # 検証データに対する予測値を求める
    va_pred = model.predict(X_valid)

    mse = mean_squared_error(Y_valid, va_pred)
    rmse = np.sqrt(mse) # RSME = √MSEの算出
    eval_metric = rmse

    print(f"eval's rmse: {eval_metric}")

    #テストデータに対する予測値を求める
    te_pred = np.array(model.predict(X_test))

    return va_pred, te_pred, model

def xgboost(X_train, Y_train, X_valid, Y_valid, X_test):

    xgb_params = {
      'objective': 'reg:linear',
      'eval_metric': 'rmse',
      # "verbosity": 0,
      "seed": SEED,
      "eta": 0.01,
      "num_boost_round": 10000,
      # "early_stopping_rounds": 10,
      # "verbose_eval": 100,
    }

    lgb_train = xgb.DMatrix(X_train, label=Y_train)
    lgb_valid = xgb.DMatrix(X_valid, label=Y_valid)
    lgb_test = xgb.DMatrix(X_test)
    evals = [(lgb_train, 'train'), (lgb_valid, 'eval')]
    evals_result = {}

    model = xgb.train(xgb_params,
                    lgb_train,
                    evals=evals,
                    evals_result=evals_result,
                    num_boost_round=10000,
                    early_stopping_rounds=10,
                    verbose_eval=1000,
                  )
  
    # 検証データに対する予測値を求める
    va_pred = model.predict(lgb_valid)

    #テストデータに対する予測値を求める
    te_pred = list(model.predict(lgb_test))

    return va_pred, te_pred, model

# 学習と予測

最初に学習済みモデル保存用にフォルダを作成しておく

In [123]:

main_df = df

scores = []
ids = []
submission = []

OUTPUT = "~/Desktop/sony_cup"

for i, (tr,va,te) in enumerate(TimeSeriesSplitGenerator(slide = True).split(main_df)):
    train_index, valid_index, test_index = tr,va,te

    print("-------------------------------------------")
    print("train:", main_df[train_index].date.min(), main_df[train_index].date.max())
    print("valid:", main_df[valid_index].date.min(), main_df[valid_index].date.max())

    train = main_df[train_index].dropna(subset=["pm25_mid"])
    valid = main_df[valid_index].dropna(subset=["pm25_mid"])
    
    if i == 0:
        year = main_df[valid_index].date.min().year
        month = main_df[valid_index].date.max().month
        test = df_test[df_test["year"] == year]
        test = test[test["month"] == month]
        print("test: ", test.date.min(), test.date.max())
        print(f'Fold : {i}')
    else:
        year = (main_df[valid_index].date.min()  + relativedelta(months=1)).year
        month = (main_df[valid_index].date.max() + relativedelta(months=1)).month
        test = df_test[df_test["year"] == year]
        test = test[test["month"] == month]
        print("test: ", test.date.min(), test.date.max())
        print(f'Fold : {i}')
    
    rmses = []
    # 予測対象日について、一日ずつモデルを作成していく
    for start_date in test["date"].unique():
        train_add = train

        X_train, Y_train = train_add.drop(columns=["pm25_mid","date","id"]), train_add["pm25_mid"]
        X_valid, Y_valid = valid.drop(columns=["pm25_mid","date","id"]), valid["pm25_mid"]
        ids += list(test[test["date"] == start_date]["id"])
        X_test = test[test["date"] == start_date].drop(columns=["date","id","year","month","day"])


        va_pred1, te_pred1, model = lightgbm(X_train, Y_train, X_valid, Y_valid, X_test)
        #pickle.dump(model, open(f"{OUTPUT}/lgbm/{start_date}.pkl", 'wb'))
        
        #va_pred2, te_pred2, model = catboost(X_train, Y_train, X_valid, Y_valid, X_test)
        #pickle.dump(model, open(f"{OUTPUT}/cat/{start_date}.pkl", 'wb'))
        
        #va_pred3, te_pred3, model = xgboost(X_train, Y_train, X_valid, Y_valid, X_test)
        #pickle.dump(model, open(f"{OUTPUT}/xgb/{start_date}.pkl", 'wb'))
      
        #va_pred = (va_pred1 + va_pred2 + va_pred3) / 3
        #te_pred = (te_pred1 + te_pred2 + te_pred3) / 3
        
        va_pred = (va_pred1)
        te_pred = (te_pred1)

        # RSME = √MSEの算出
        mse = mean_squared_error(Y_valid, va_pred)
        rmse = np.sqrt(mse)

        rmses.append(rmse)

        #テストデータに対する予測値を求める
        submission += list(te_pred)
        print('')
        print(f"Fold: {i} {start_date} RMSE:{rmse}")
        print("")

    # フォールド毎の検証時のスコアを格納
    scores.append(np.mean(rmses))

    print('')
    print('################################')
    print(f"Fold: {i} RMSE:{np.mean(rmses)}")
    print("")

print() 
print(f"CV: {np.mean(scores)}")

# CV: 3.4555098

-------------------------------------------
train: 2019-01-01 00:00:00 2019-01-31 00:00:00
valid: 2019-01-01 00:00:00 2019-01-31 00:00:00
test:  2019-01-01 00:00:00 2019-01-31 00:00:00
Fold : 0
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001218 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12155
[LightGBM] [Info] Number of data points in the train set: 5494, number of used features: 50
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Info] Start training from score 72.650419
Training until validat

[1000]	train's rmse: 14.8014	valid's rmse: 14.8014
[2000]	train's rmse: 9.70757	valid's rmse: 9.70757
[3000]	train's rmse: 6.68055	valid's rmse: 6.68055
[4000]	train's rmse: 4.70215	valid's rmse: 4.70215
[5000]	train's rmse: 3.36295	valid's rmse: 3.36295
[6000]	train's rmse: 2.43026	valid's rmse: 2.43026
[7000]	train's rmse: 1.75877	valid's rmse: 1.75877
[8000]	train's rmse: 1.27838	valid's rmse: 1.27838
[9000]	train's rmse: 0.937141	valid's rmse: 0.937141
[10000]	train's rmse: 0.688711	valid's rmse: 0.688711
Did not meet early stopping. Best iteration is:
[10000]	train's rmse: 0.688711	valid's rmse: 0.688711

Fold: 0 2019-01-06T00:00:00.000000000 RMSE:0.6887106600931876

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-c


Fold: 0 2019-01-11T00:00:00.000000000 RMSE:0.6887106600931876

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001493 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12155
[LightGBM] [Info] Number of data points in the train set: 5494, number of used features: 50
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Info] Start training from score 72.650419
Training until validation scores don't improve for 10 rounds
[1000]	train's rmse: 14.8014	valid's rmse: 14.8014
[2000]	train's rmse: 9.70757	valid's rms

[1000]	train's rmse: 14.8014	valid's rmse: 14.8014
[2000]	train's rmse: 9.70757	valid's rmse: 9.70757
[3000]	train's rmse: 6.68055	valid's rmse: 6.68055
[4000]	train's rmse: 4.70215	valid's rmse: 4.70215
[5000]	train's rmse: 3.36295	valid's rmse: 3.36295
[6000]	train's rmse: 2.43026	valid's rmse: 2.43026
[7000]	train's rmse: 1.75877	valid's rmse: 1.75877
[8000]	train's rmse: 1.27838	valid's rmse: 1.27838
[9000]	train's rmse: 0.937141	valid's rmse: 0.937141
[10000]	train's rmse: 0.688711	valid's rmse: 0.688711
Did not meet early stopping. Best iteration is:
[10000]	train's rmse: 0.688711	valid's rmse: 0.688711

Fold: 0 2019-01-17T00:00:00.000000000 RMSE:0.6887106600931876

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-c


Fold: 0 2019-01-22T00:00:00.000000000 RMSE:0.6887106600931876

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001370 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12155
[LightGBM] [Info] Number of data points in the train set: 5494, number of used features: 50
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Info] Start training from score 72.650419
Training until validation scores don't improve for 10 rounds
[1000]	train's rmse: 14.8014	valid's rmse: 14.8014
[2000]	train's rmse: 9.70757	valid's rms

...

[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005512 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12217
[LightGBM] [Info] Number of data points in the train set: 51933, number of used features: 50
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Info] Start training from score 57.123848
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[956]	train's rmse: 18.8505	valid's rmse: 22.6516

Fold: 10 2019-11-13T00:00:00.000000000 RMSE:22.651645303113128

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[Lig

Early stopping, best iteration is:
[956]	train's rmse: 18.8505	valid's rmse: 22.6516

Fold: 10 2019-11-21T00:00:00.000000000 RMSE:22.651645303113128

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005600 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12217
[LightGBM] [Info] Number of data points in the train set: 51933, number of used features: 50
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Info] Start training from score 57.123848
Training until validation scores don't improve for 10 rounds
Earl

[LightGBM] [Info] Total Bins 12217
[LightGBM] [Info] Number of data points in the train set: 51933, number of used features: 50
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Info] Start training from score 57.123848
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[956]	train's rmse: 18.8505	valid's rmse: 22.6516

Fold: 10 2019-11-30T00:00:00.000000000 RMSE:22.651645303113128


################################
Fold: 10 RMSE:22.651645303113128

-------------------------------------------
train: 2019-01-01 00:00:00 2019-10-31 00:00:00
valid: 2019-11-01 00:00:00 2019-11-30 00:00:00
test:  2019-12-01 00:00:00 2019-12-31 00:00:00
Fold : 11
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval

[1000]	train's rmse: 19.057	valid's rmse: 22.3347
Early stopping, best iteration is:
[1091]	train's rmse: 18.8742	valid's rmse: 22.2664

Fold: 11 2019-12-08T00:00:00.000000000 RMSE:22.26642333679352

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005044 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12216
[LightGBM] [Info] Number of data points in the train set: 57928, number of used features: 50
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Info] Start training from score 57.690480
Training until 

KeyboardInterrupt: 

In [ ]:
sub = pd.Series(submission)
sub.index = ids
sub.to_csv(f"{OUTPUT}/submmission_ensenble(day_by_day).csv", header=False)